In [1]:
DATA_NAME = 'coco-outdoor' 
TRANSFORM = 'wavelet-vertical'
CHANNEL = 'gray'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_gray_vertical_wavelet_outdoor_coco.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
    cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9
2,3000.91,-0.01,0.00,-0.01,-0.01,-0.00,-0.00,0.00
3,-0.01,820.13,-0.00,0.01,0.00,0.00,0.00,-0.00
4,0.00,-0.00,189.17,-0.00,-0.00,0.00,-0.00,0.00
5,-0.01,0.01,-0.00,42.68,0.00,0.00,0.00,-0.00
6,-0.01,0.00,-0.00,0.00,9.07,-0.00,0.00,0.00
7,-0.00,0.00,0.00,0.00,-0.00,1.88,-0.00,-0.00
8,-0.00,0.00,-0.00,0.00,0.00,-0.00,0.44,0.00
9,0.00,-0.00,0.00,-0.00,0.00,-0.00,0.00,0.12


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8,9
2,1.00000,-0.00001,0.00000,-0.00003,-0.00004,-0.00002,-0.00004,0.00001
3,-0.00001,1.00000,-0.00000,0.00003,0.00001,0.00003,0.00000,-0.00003
4,0.00000,-0.00000,1.00000,-0.00004,-0.00002,0.00006,-0.00002,0.00003
5,-0.00003,0.00003,-0.00004,1.00000,0.00002,0.00000,0.00002,-0.00004
6,-0.00004,0.00001,-0.00002,0.00002,1.00000,-0.00001,0.00003,0.00006
7,-0.00002,0.00003,0.00006,0.00000,-0.00001,1.00000,-0.00002,-0.00002
8,-0.00004,0.00000,-0.00002,0.00002,0.00003,-0.00002,1.00000,0.00002
9,0.00001,-0.00003,0.00003,-0.00004,0.00006,-0.00002,0.00002,1.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

0.025360260075184358

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[1.12763116e+06 8.23633286e+04 4.26280883e+03 2.08394792e+02
 8.82927207e+00 3.41215265e-01 1.46584821e-02 1.78703727e-34]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7
0,0.999073,-0.042056,-0.008912,-0.002006,-0.000428,-0.000089,-0.000021,-0.000006
1,0.041639,0.998255,-0.040910,-0.008678,-0.001838,-0.000381,-0.000089,-0.000025
2,0.010487,0.040026,0.997951,-0.047814,-0.009564,-0.001973,-0.000463,-0.000127
3,0.002831,0.010360,0.046775,0.997220,-0.055872,-0.010977,-0.002552,-0.000711
4,0.000749,0.002732,0.011878,0.054356,0.995676,-0.072468,-0.015968,-0.004413
5,0.000203,0.000740,0.003211,0.014273,0.069114,0.989559,-0.121678,-0.031238
6,0.000066,0.000239,0.001039,0.004597,0.021672,0.107352,0.955889,-0.272507
7,0.000039,0.000143,0.000619,0.002746,0.012913,0.062225,0.266850,0.961636


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[0.0009266086171372256,
 0.0017446332428063815,
 0.0020490063851079654,
 0.002780086873869436,
 0.004324249952702197,
 0.010440845509546604,
 0.04411102562607905,
 0.038363684412880406]